In [ ]:
# 导入必要的库
from ultralytics import YOLO
import os
from pathlib import Path
import matplotlib.pyplot as plt
import random
import math
import numpy as np
import matplotlib
import torch
import cv2
from IPython.display import display, Image

# 配置 Matplotlib 显示中文
matplotlib.rcParams['font.sans-serif'] = ['PingFang SC', 'SimHei', 'Songti SC', 'Arial Unicode MS']
matplotlib.rcParams['axes.unicode_minus'] = False

# 设置 Matplotlib 内联显示，使图表直接显示在 notebook 中
%matplotlib inline

辅助函数

In [ ]:
# 从 bottle_detection_utils.py 导入，或者直接定义在这里
def box_iou(box1, box2):
    """计算两个框的IoU"""
    # 交集区域
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    # 交集面积
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    
    # 各自面积
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    # IoU
    iou = intersection / (area1 + area2 - intersection + 1e-6)
    return iou

def detect_bottle_with_neck(image_path, model, conf=0.5, device='cpu', iou=0.45):
    """
    对图像进行玻璃瓶检测，同时处理瓶颈部分的检测
    
    参数:
        image_path (str): 要检测的图像路径
        model (YOLO): 加载好的YOLO模型对象
        conf (float): 检测置信度阈值，默认0.5
        device (str): 使用的设备，如'cpu'或'cuda'（如有GPU）
        iou (float): NMS的IoU阈值，默认0.45，降低此值可减少重叠检测
        
    返回:
        list: 检测结果列表
    """
    # 运行YOLO预测
    results = model.predict(
        source=image_path,
        conf=conf,
        device=device,
        verbose=False,
        save=True,
        save_txt=False,
        save_conf=True,
        save_crop=False,
        show_labels=True,
        show_conf=True,
        visualize=False,
        augment=False,
        iou=iou  # 添加IoU阈值参数
    )
    
    # 后处理：修正椭圆形截面误检为瓶口的问题
    for i, result in enumerate(results):
        if len(result.boxes) > 0:
            # 先处理类别修正问题（将小面积椭圆形截面修正为瓶口）
            for j, box in enumerate(result.boxes):
                cls = box.cls[0].cpu().item()
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                
                # 计算框的面积和宽高比
                area = (x2 - x1) * (y2 - y1)
                aspect_ratio = (x2 - x1) / (y2 - y1) if (y2 - y1) > 0 else 0
                
                # 修正椭圆形截面的玻璃瓶误检为瓶口
                # 类别索引2是椭圆形截面的玻璃瓶，类别索引1是瓶口
                if cls == 2 and area < 0.1 and aspect_ratio >= 0.8 and aspect_ratio <= 1.2:
                    # 将小面积且接近正方形的椭圆形截面修正为瓶口
                    # 注意这里要直接修改YOLO结果的张量
                    box.cls[0] = torch.tensor([1.0], device=box.cls.device)
            
            # 如果同一区域有多个类别检测，只保留最高置信度的类别
            if len(result.boxes) > 1:
                # 将所有检测框转换为格式 [x1, y1, x2, y2, conf, cls]
                boxes = []
                for j, box in enumerate(result.boxes):
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    conf = box.conf[0].cpu().numpy()
                    cls = box.cls[0].cpu().numpy()
                    boxes.append([x1, y1, x2, y2, conf, cls])
                
                boxes = np.array(boxes)
                
                # 按置信度排序（降序）
                sort_idx = np.argsort(-boxes[:, 4])
                boxes = boxes[sort_idx]
                
                # 特殊规则：瓶口类别(类别索引1)优先保留
                # 其他类别(0,2,3)之间如果有重叠则只保留置信度最高的一个
                bottle_types = []  # 保存非瓶口类别的索引
                bottle_mouth = []  # 保存瓶口类别的索引
                
                # 先分类
                for j in range(len(boxes)):
                    if boxes[j, 5] == 1:  # 瓶口类别
                        bottle_mouth.append(j)
                    else:  # 瓶身类别
                        bottle_types.append(j)
                        
                # 处理瓶身类别，保留最高置信度且无明显重叠的
                kept_bottle_types = []
                for j in bottle_types:
                    should_keep = True
                    box1 = boxes[j, :4]
                    cls1 = boxes[j, 5]
                    
                    for k in kept_bottle_types:
                        box2 = boxes[k, :4]
                        iou_val = box_iou(box1, box2)
                        # 如果重叠度较高，则不保留当前框
                        if iou_val > 0.3:  # 降低IoU阈值以更严格地过滤重叠框
                            should_keep = False
                            break
                            
                    if should_keep:
                        kept_bottle_types.append(j)
                
                # 合并瓶口类别和保留的瓶身类别
                kept_indices = kept_bottle_types + bottle_mouth
                
                # 重建结果
                if len(kept_indices) < len(boxes):
                    # 需要过滤结果
                    new_boxes = result.boxes[0:0]  # 创建空的Boxes对象
                    
                    for idx in kept_indices:
                        new_boxes.append(result.boxes[sort_idx[idx]])
                    
                    # 替换原始结果
                    result.boxes = new_boxes
    
    return results

配置参数

In [ ]:
# 配置参数（替代命令行参数）
# 可根据需要调整这些参数
CONFIG = {
    # 训练相关
    "SKIP_TRAINING": False,  # 设置为 True 跳过训练，直接使用已有模型
    "DATA_YAML": "data.yaml",  # 数据集配置文件
    "DATASET_DIR": "./检测 玻璃瓶",  # 数据集路径
    "DEVICE": "mps",  # 设备：'cpu'、'cuda'（NVIDIA GPU）或 'mps'（Apple Silicon）
    "EPOCHS": 150,  # 训练轮数
    "BATCH_SIZE": 4,  # 批量大小
    "IMAGE_SIZE": 1280,  # 输入图像大小
    "CONF_THRESHOLD": 0.5,  # 置信度阈值
    
    # 数据增强参数
    "AUGMENT": True,  # 是否使用数据增强
    "DEGREES": 180.0,  # 旋转增强角度范围 -180 到 +180 度
    "MOSAIC": 1.0,  # Mosaic 增强
    "FLIPUD": 0.5,  # 上下翻转概率
    "FLIPLR": 0.5,  # 左右翻转概率
    "SCALE": 0.5,  # 尺度增强
    
    # 预测和评估相关
    "MAX_IMAGES": 20,  # 最多处理的图片数量
    "SAVE_DIR": "runs/detect",  # 保存目录
    
    # 其他
    "MODEL_PATH": None,  # 模型路径，None 表示使用默认路径
}

# 打印当前配置
for key, value in CONFIG.items():
    print(f"{key}: {value}")

模型初始化

In [ ]:
# 初始化模型
print("加载模型结构...")
model = YOLO('yolo11n.pt')  # 使用 YOLOv11n 模型
best_model = None  # 初始化 best_model 变量，将在训练后或加载现有模型时赋值

# 如果不训练，尝试加载现有的最佳模型
if CONFIG["SKIP_TRAINING"]:
    print("\n跳过训练，尝试加载现有的最佳模型...")
    
    # 如果指定了模型路径，则使用指定路径
    if CONFIG["MODEL_PATH"] and os.path.exists(CONFIG["MODEL_PATH"]):
        best_model_path = CONFIG["MODEL_PATH"]
    else:
        # 查找最新的训练目录
        try:
            runs_dir = 'runs/detect'
            all_train_dirs = sorted([os.path.join(runs_dir, d) for d in os.listdir(runs_dir) 
                                   if d.startswith('train') and os.path.isdir(os.path.join(runs_dir, d))])
            if all_train_dirs:
                best_model_path = os.path.join(all_train_dirs[-1], 'weights/best.pt')
                print(f"自动检测到最新的训练模型：{best_model_path}")
            else:
                print("错误：找不到训练模型，将使用初始模型")
                best_model_path = 'yolo11n.pt'
        except Exception as e:
            print(f"查找模型时出错: {e}")
            best_model_path = 'yolo11n.pt'
    
    # 加载模型
    try:
        print(f"加载模型：{best_model_path}")
        best_model = YOLO(best_model_path)
    except Exception as e:
        print(f"加载模型时出错: {e}")

训练模型

In [ ]:
# 训练模型（如果未跳过）
results = None  # 初始化 results 变量

if not CONFIG["SKIP_TRAINING"]:
    print("\n--- 开始训练模型 ---")
    try:
        print(f"开始训练 (epochs={CONFIG['EPOCHS']}, device={CONFIG['DEVICE']}, ...)...")
        
        # 使用增强的数据训练模型
        results = model.train(
            data=CONFIG["DATA_YAML"],
            epochs=CONFIG["EPOCHS"],
            device=CONFIG["DEVICE"],
            imgsz=CONFIG["IMAGE_SIZE"],
            batch=CONFIG["BATCH_SIZE"],
            patience=70,  # 早停耐心值
            augment=CONFIG["AUGMENT"],
            mosaic=CONFIG["MOSAIC"],
            flipud=CONFIG["FLIPUD"],
            fliplr=CONFIG["FLIPLR"],
            scale=CONFIG["SCALE"],
            degrees=CONFIG["DEGREES"],  # 旋转增强
            hsv_h=0.015,  # 色调增强
            hsv_s=0.7,    # 饱和度增强
            hsv_v=0.4,    # 亮度增强
            resume=False,
            overlap_mask=True,   # 使用重叠掩码提高精度
            single_cls=False,    # 多类别检测
        )
        
        print("训练完成！")
        
        # 训练完成后，将最佳模型赋值给 best_model
        if hasattr(results, 'save_dir') and results.save_dir:
            best_model_path = os.path.join(results.save_dir, 'weights/best.pt')
            print(f"加载训练后的最佳模型：{best_model_path}")
            best_model = YOLO(best_model_path)
    
    except Exception as e:
        print(f"训练过程中发生错误: {e}")

验证模型

In [ ]:
# 验证最佳模型
if best_model is not None:
    print("\n--- 开始验证模型 ---")
    try:
        # 设置字体以确保中文显示正确
        matplotlib.rcParams['font.sans-serif'] = ['PingFang SC', 'SimHei', 'Songti SC', 'Arial Unicode MS']
        
        # 运行验证
        print("运行验证...")
        metrics = best_model.val(data=CONFIG["DATA_YAML"])
        
        # 打印总体指标
        print(f"验证结果 mAP50-95: {metrics.box.map:.4f}")
        print(f"验证结果 mAP50:    {metrics.box.map50:.4f}")
        print(f"验证结果 mAP75:    {metrics.box.map75:.4f}")
        
        # 可视化各类别 mAP50-95
        maps_per_category = metrics.box.maps
        class_names = best_model.names  # 从加载的模型获取类别名称
        
        if maps_per_category is not None and class_names:
            if isinstance(maps_per_category, (list, np.ndarray)):
                maps_values = np.array(maps_per_category)
                sorted_class_names = [class_names[i] for i in range(len(maps_values))]
                
                print("\n各类别 mAP50-95:")
                for name, val in zip(sorted_class_names, maps_values):
                    print(f"  {name}: {val:.4f}")
                
                # 创建条形图
                plt.figure(figsize=(max(6, len(sorted_class_names) * 1.2), 6))
                bars = plt.bar(sorted_class_names, maps_values, color='skyblue')
                plt.xlabel("类别")
                plt.ylabel("mAP50-95")
                plt.title("各类别 mAP50-95 评估结果 (验证集)")
                plt.ylim(0, 1.05)  # 留一点顶部空间
                plt.xticks(rotation=45, ha='right')  # 旋转标签以防重叠
                
                # 在条形图上显示数值
                for bar in bars:
                    yval = bar.get_height()
                    plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.01, f'{yval:.3f}', 
                             va='bottom', ha='center', fontsize=9)
                
                plt.tight_layout(pad=1.5)  # 调整布局
                plt.show()  # 显示图表
            else:
                print("警告：验证结果中无法获取有效的各类别 mAP 数据进行可视化。")
    
    except Exception as e:
        print(f"验证过程中发生错误: {e}")

测试单张图片

In [ ]:
# 测试模型预测单张图片的功能
def predict_single_image(image_path, model=None, conf=None, show_result=True):
    """
    预测单张图片并显示结果
    
    参数:
        image_path (str): 图片路径
        model: 使用的模型，如果为 None 则使用 best_model
        conf (float): 置信度阈值，如果为 None 则使用 CONFIG["CONF_THRESHOLD"]
        show_result (bool): 是否显示结果图片
    """
    if not os.path.exists(image_path):
        print(f"错误：找不到图片: {image_path}")
        return None
    
    # 使用默认值
    if model is None:
        model = best_model
    if conf is None:
        conf = CONFIG["CONF_THRESHOLD"]
    
    if model is None:
        print("错误：模型未加载")
        return None
    
    print(f"预测图片: {image_path}")
    
    try:
        # 使用检测函数
        results = detect_bottle_with_neck(
            image_path=image_path,
            model=model,
            conf=conf,
            device=CONFIG["DEVICE"]
        )
        
        # 打印结果
        for i, r in enumerate(results):
            print(r.verbose())
            
        # 显示结果图片
        if show_result and len(results) > 0:
            # 找到保存的结果图片
            save_dir = results[0].save_dir
            img_name = Path(image_path).name
            result_path = os.path.join(save_dir, img_name)
            
            if os.path.exists(result_path):
                plt.figure(figsize=(12, 8))
                img = plt.imread(result_path)
                plt.imshow(img)
                plt.axis('off')
                plt.title(f"检测结果: {img_name}")
                plt.show()
            else:
                print(f"警告：找不到结果图片: {result_path}")
                # 直接显示原图
                plt.figure(figsize=(12, 8))
                img = plt.imread(image_path)
                plt.imshow(img)
                plt.axis('off')
                plt.title(f"原图: {img_name}")
                plt.show()
        
        return results
    
    except Exception as e:
        print(f"预测过程中发生错误: {e}")
        return None

# 测试函数 - 可以选择一张图片进行测试
# 取消下面的注释并提供图片路径来测试
# test_image_path = "检测 玻璃瓶/水滴形截面的玻璃瓶/Image_20191006232558011.jpg"
# predict_single_image(test_image_path)

批量测试

# 在多张图片上测试模型
def test_on_multiple_images(num_images=5, conf=None, selected_category=None):
    """
    在数据集中随机选择图片测试模型
    
    参数:
        num_images (int): 要测试的图片数量
        conf (float): 置信度阈值，如果为 None 则使用 CONFIG["CONF_THRESHOLD"]
        selected_category (str): 只从特定类别目录中选择图片，如果为 None 则从所有目录选择
    """
    if best_model is None:
        print("错误：模型未加载")
        return
    
    if conf is None:
        conf = CONFIG["CONF_THRESHOLD"]
    
    # 使用全局 dataset_dir
    source_base_dir = CONFIG["DATASET_DIR"]
    
    # 定义原始类别目录
    original_categories = ["水滴形截面的玻璃瓶", "椭圆形截面的玻璃瓶", "圆形截面的玻璃瓶"]
    
    if selected_category is not None and selected_category not in original_categories:
        print(f"警告：类别 '{selected_category}' 不存在。可用类别: {original_categories}")
        return
    
    # 需要查找的目录
    search_categories = [selected_category] if selected_category else original_categories
    
    # 收集图片
    image_files = []
    
    for category in search_categories:
        category_path = os.path.join(source_base_dir, category)
        try:
            if os.path.exists(category_path):
                category_images = [os.path.join(category_path, f)
                                  for f in os.listdir(category_path)
                                  if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
                
                if category_images:
                    image_files.extend(category_images)
                    print(f"从 '{category}' 找到 {len(category_images)} 张图片")
            else:
                print(f"警告：找不到目录 {category_path}")
        except Exception as e:
            print(f"查找图片时出错 ({category_path}): {e}")
    
    # 增加检测 mini 目录下的图片
    mini_images_path = os.path.join(source_base_dir, "mini/images")
    try:
        if os.path.exists(mini_images_path) and (selected_category is None):
            mini_images = [os.path.join(mini_images_path, f)
                           for f in os.listdir(mini_images_path)
                           if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
            
            if mini_images:
                image_files.extend(mini_images)
                print(f"从 'mini/images' 找到 {len(mini_images)} 张图片")
    except Exception as e:
        print(f"查找 mini 目录图片时出错: {e}")
    
    if not image_files:
        print(f"错误：未找到任何图片")
        return
    
    # 随机选择图片
    if len(image_files) > num_images:
        print(f"从 {len(image_files)} 张图片中随机选择 {num_images} 张")
        random.shuffle(image_files)
        image_files = image_files[:num_images]
    
    print(f"\n将对 {len(image_files)} 张图片进行预测...")
    
    # 预测每张图片并显示结果
    for i, image_path in enumerate(image_files):
        print(f"\n[{i+1}/{len(image_files)}] 处理图片: {os.path.basename(image_path)}")
        predict_single_image(image_path, conf=conf)

# 测试函数 - 可以选择运行
# 取消下面的注释来测试模型在多张图片上的表现
# test_on_multiple_images(num_images=3)
# test_on_multiple_images(num_images=2, selected_category="水滴形截面的玻璃瓶")

特定角度图片测试

In [ ]:
# 测试模型在旋转瓶子图片上的表现
# 这是为了测试我们添加的旋转数据增强是否有效

# 指定要测试的旋转瓶子图片
rotated_bottle_images = [
    "检测 玻璃瓶/水滴形截面的玻璃瓶/Image_20191006232145535.jpg",  # 原始侧放瓶子
    "检测 玻璃瓶/水滴形截面的玻璃瓶/Image_20191006232558011.jpg",  # 原始侧放瓶子
    "检测 玻璃瓶/水滴形截面的玻璃瓶/Image_20191007032013552.jpg"   # 旋转后瓶口朝上的瓶子
]

# 测试每张图片
print("测试模型在旋转瓶子图片上的表现...")
for img_path in rotated_bottle_images:
    if os.path.exists(img_path):
        predict_single_image(img_path)
    else:
        print(f"找不到图片: {img_path}")